In [2]:
import re
#pip和环境（需要修改模型地址）

import subprocess
# 安装包
subprocess.run(['pip', 'install', 'adversarial-robustness-toolbox'])

input_base="data"

labelselect=[1,7,8,12,18,21,47,84,88,94,97,107,112,196,208,231,283,301,309,310,327,508,650,656,665,668,671,892,928,937]
# 读取文件内容
with open('node.txt', 'r', encoding='utf-8') as file:
    content = file.read()

# 解析数据
data = []
pattern = r'\((\d+),\s*(\d+)\)'
matches = re.findall(pattern, content)

for match in matches:
    x = int(match[0])
    y = int(match[1])
    data.append((x, y))

print(data[0])
import re


input_base="data"

labelselect=[1,7,8,12,18,21,47,84,88,94,97,107,112,196,208,231,283,301,309,310,327,508,650,656,665,668,671,892,928,937]
# 读取文件内容
with open('node.txt', 'r', encoding='utf-8') as file:
    content = file.read()

# 解析数据
data = []
pattern = r'\((\d+),\s*(\d+)\)'
matches = re.findall(pattern, content)

for match in matches:
    x = int(match[0])
    y = int(match[1])
    data.append((x, y))

print(data[0])


(51, 1)
(51, 1)


In [3]:
def organize_files_by_label(input_file_path):
    # 创建一个字典来存储按标签分类的文件
    organized_data = {}

    try:
        # 读取输入文件
        with open(input_file_path, 'r') as file:
            for line in file:
                # 移除行末的换行符并按空格分割
                parts = line.strip().split()

                if len(parts) >= 2:
                    filename = parts[0]
                    label = parts[1]

                    # 如果标签已存在于字典中，添加文件名到对应的列表
                    if label in organized_data:
                        organized_data[label].append(filename)
                    else:
                        # 否则，创建一个新的列表
                        organized_data[label] = [filename]
                else:
                    print(f"跳过无效行: {line.strip()}")

        # 按标签的数值大小排序
        sorted_labels = sorted(organized_data.keys(), key=lambda x: int(x))

        """# 打印结果
        print("整理后的文件:")
        for label in sorted_labels:
            print(f"标签 {label}:")
            for filename in organized_data[label]:
                print(f"  {filename}")
            print()"""

        # 如果需要保存结果到文件，可以取消以下注释
        # save_organized_data(sorted_labels, organized_data)

    except FileNotFoundError:
        print(f"错误: 无法找到文件 '{input_file_path}'")
    except Exception as e:
        print(f"处理文件时出错: {e}")
    return organized_data

# 如果需要保存结果到文件，可以使用以下函数
def save_organized_data(sorted_labels, organized_data, output_file_path='organized_files.txt'):
    with open(output_file_path, 'w') as file:
        for label in sorted_labels:
            file.write(f"标签 {label}:\n")
            for filename in organized_data[label]:
                file.write(f"  {filename}\n")
            file.write("\n")
    print(f"已将结果保存到文件: {output_file_path}")
    
    
input_file = "val.txt"  
sort=organize_files_by_label(input_file)
# 将所有的.png改为.JPEG
for label in sort:
    sort[label] = [filename.replace('.png', '.JPEG') for filename in sort[label]]

In [4]:
import os
import torch
from PIL import Image
from torchvision import transforms, models

# 检查CUDA可用性
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def find_max_activation_image(target_label, layer_idx, node_idx, model, sort, input_base, preprocess):
    # 获取目标标签对应的文件名列表
    filenames = sort.get(str(target_label), [])
    if not filenames:
        raise ValueError(f"No images found for label {target_label}")
    
    # 构建完整路径列表
    image_paths = [os.path.join(input_base, fname) for fname in filenames]
    
    # 收集模型层（确保模型在GPU）
    model = model.to(device)
    layers = []
    def collect_layers(module, prefix=""):
        for name, child in module.named_children():
            if len(list(child.children())) == 0:
                layers.append((f"{prefix}{name}", child))
            else:
                collect_layers(child, prefix=f"{prefix}{name}.")
    collect_layers(model)
    
    # 注册钩子（自动处理GPU数据）
    target_layer = layers[layer_idx][1]
    activation = {}
    def hook(module, input, output):
        activation['output'] = output.detach().to(device)  # 确保输出在GPU
    handle = target_layer.register_forward_hook(hook)
    
    # 预处理参数（创建在GPU）
    white_value = torch.tensor([
        (1.0 - 0.485)/0.229,
        (1.0 - 0.456)/0.224,
        (1.0 - 0.406)/0.225
    ], device=device).view(1, 3, 1, 1)  # 添加device参数
    
    inv_normalize = transforms.Normalize(
        mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
        std=[1/0.229, 1/0.224, 1/0.225]
    )
    
    try:
        # 创建以标签、层和节点命名的输出目录
        output_base = f"output_images_{target_label}_layer{layer_idx}_node{node_idx}"
        os.makedirs(output_base, exist_ok=True)
        
        for img_path in image_paths:
            try:
                # 图像预处理（保持数据在GPU）
                img = Image.open(img_path).convert('RGB')
                input_tensor = preprocess(img).unsqueeze(0).to(device)  # 移动到GPU
                white_tensor = torch.ones_like(input_tensor, device=device) * white_value
                
                max_activation = -float('inf')
                max_image = None
                
                # 滑动窗口生成掩码（使用CUDA加速）
                with torch.cuda.amp.autocast():  # 混合精度加速
                    flag=80
                    for i in range(0, 224 - flag + 1):
                        for j in range(0, 224 - flag + 1):
                            masked = white_tensor.clone()
                            masked[..., i:i+flag, j:j+flag] = input_tensor[..., i:i+flag, j:j+flag]
                            
                            with torch.no_grad():
                                model(masked)
                            
                            output = activation['output']
                            # 保持计算在GPU
                            if output.ndim == 4:
                                act = output[0, node_idx].mean()
                            elif output.ndim == 2:
                                act = output[0, node_idx]
                            else:
                                continue
                            
                            # 比较前移到GPU
                            if act > max_activation:
                                max_activation = act
                                # 延迟数据回传
                                processed_img = inv_normalize(masked.squeeze().cpu())
                                max_image = transforms.ToPILImage()(processed_img)
                                
                                # 释放中间变量内存
                                del masked, output
                                torch.cuda.empty_cache()           
                
                # 保存结果
                if max_image:
                    os.makedirs(output_base, exist_ok=True)
                    output_path = os.path.join(output_base, os.path.basename(img_path))
                    max_image.save(output_path)
                    print(f"Saved result for {img_path} to {output_path}")
                
            except Exception as e:
                print(f"Error processing {img_path}: {e}")
            finally:
                # 显式释放大内存
                del input_tensor, white_tensor
                torch.cuda.empty_cache()
    finally:
        handle.remove()

# 初始化模型（直接在GPU创建）
model = models.resnet18(pretrained=True).to(device)
model.eval()

# 预处理保持数据在GPU
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                       [0.229, 0.224, 0.225])
])


Using device: cuda


In [7]:
16/255

0.06274509803921569

In [9]:
import re
import os
import torch
import numpy as np
from PIL import Image
from torchvision import transforms, models
from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import (
    FastGradientMethod,
    ProjectedGradientDescent,
    CarliniL2Method,
    BasicIterativeMethod
)

# 配置参数
class Config:
    input_base = "data"
    labelselect = [1,7,8,12,18,21,47,84,88,94,97,107,112,196,208,231,283,301,309,310,327,508,650,656,665,668,671,892,928,937]  # 目标处理标签
    node_file = "node.txt"
    val_file = "val.txt"
    adv_root = "adversarials"  # 对抗样本存储根目录
    result_root = "results"    # 分析结果存储根目录
    window_size = 80          # 敏感区域分析窗口大小
    window_stride = 1        # 滑动步长（平衡速度与精度）

# 攻击方法配置字典
ATTACK_CONFIGS = {
    'FGSM': {
        'class': FastGradientMethod,
        'params': {'eps': 0.08}
    },
    'PGD': {
        'class': ProjectedGradientDescent,
        'params': {'eps': 0.1, 'max_iter': 40}
    },
    'CW': {
        'class': CarliniL2Method,
        'params': {'confidence': 0.5}
    },
    'BIM': {
        'class': BasicIterativeMethod,
        'params': {'eps': 0.05, 'max_iter': 10}
    }
}
# 数据预处理管道
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                     std=[0.229, 0.224, 0.225])
])

# 逆归一化转换
inv_normalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
    std=[1/0.229, 1/0.224, 1/0.225]
)

def setup_device():
    """初始化硬件设备"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    return device

def load_node_mapping():
    """加载节点映射关系"""
    with open(Config.node_file, 'r', encoding='utf-8') as f:
        content = f.read()
    return [tuple(map(int, m)) for m in re.findall(r'\((\d+),\s*(\d+)\)', content)]

def filter_target_labels(organized_data):
    """过滤出目标标签数据"""
    target_labels = set(map(str, Config.labelselect))
    return {k: v for k, v in organized_data.items() if k in target_labels}

def create_adv_classifier(model, device):
    """创建ART分类器"""
    return PyTorchClassifier(
        model=model,
        clip_values=(0, 1),
        loss=torch.nn.CrossEntropyLoss(),
        optimizer=torch.optim.Adam(model.parameters(), lr=0.001),
        input_shape=(3, 224, 224),
        nb_classes=1000,
        device_type='gpu' if str(device) == 'cuda' else 'cpu'
    )

def generate_adversarial_batch(model, device):
    """批量生成对抗样本"""
    model.eval()
    
    # 组织原始数据并过滤
    organized = organize_files_by_label(Config.val_file)
    filtered_data = filter_target_labels(organized)
    
    for attack_name, config in ATTACK_CONFIGS.items():
        print(f"\nGenerating {attack_name} adversarials...")
        
        # 初始化攻击方法
        classifier = create_adv_classifier(model, device)
        AttackClass = config['class']
        attack = AttackClass(classifier, **config['params'])
        
        # 遍历每个目标标签
        for label_str, filenames in filtered_data.items():
            label_dir = os.path.join(Config.adv_root, attack_name, label_str)
            os.makedirs(label_dir, exist_ok=True)
            
            # 处理每个图像
            for fname in filenames:
                src_path = os.path.join(Config.input_base, fname)
                dst_path = os.path.join(label_dir, os.path.basename(fname))
                
                if os.path.exists(dst_path):
                    continue  # 跳过已处理文件
                
                try:
                    img = Image.open(src_path).convert('RGB')
                    tensor = preprocess(img).unsqueeze(0).to(device)
                    
                    # 生成对抗样本
                    x_adv = attack.generate(tensor.cpu().numpy())
                    adv_tensor = torch.from_numpy(x_adv).to(device)
                    
                    # 保存结果
                    adv_img = transforms.ToPILImage()(inv_normalize(adv_tensor.squeeze().cpu()))
                    adv_img.save(dst_path)
                    
                except Exception as e:
                    print(f"Error processing {fname}: {str(e)}")

def analyze_activations(model, device, node_mapping):
    """分析对抗样本激活区域"""
    model.eval()
    
    # 构建标签-节点映射字典
    label_to_node = {
        str(label): node_mapping[i] 
        for i, label in enumerate(Config.labelselect)
    }
    
    for attack_name in ATTACK_CONFIGS.keys():
        print(f"\nAnalyzing {attack_name} samples...")
        
        # 遍历每个目标标签
        for label_str in map(str, Config.labelselect):
            layer_idx, node_idx = label_to_node[label_str]
            
            # 准备路径
            src_dir = os.path.join(Config.adv_root, attack_name, label_str)
            dst_dir = os.path.join(
                Config.result_root,
                attack_name,
                f"label_{label_str}_layer{layer_idx}_node{node_idx}"
            )
            os.makedirs(dst_dir, exist_ok=True)
            
            # 注册钩子
            activation = {}
            handle = register_activation_hook(model, layer_idx, activation)
            
            try:
                # 处理每个对抗样本
                for fname in os.listdir(src_dir):
                    if not fname.endswith('.JPEG'):
                        continue
                    
                    img_path = os.path.join(src_dir, fname)
                    output_path = os.path.join(dst_dir, fname)
                    
                    if os.path.exists(output_path):
                        continue
                    
                    try:
                        analyze_single_image(
                            model=model,
                            img_path=img_path,
                            node_idx=node_idx,
                            activation=activation,
                            output_path=output_path,
                            device=device
                        )
                    except Exception as e:
                        print(f"Error analyzing {fname}: {str(e)}")
                        
            finally:
                handle.remove()

def register_activation_hook(model, layer_idx, activation):
    """注册前向钩子捕获激活值"""
    layers = []
    def _collect_layers(module):
        if not list(module.children()):
            layers.append(module)
        else:
            for child in module.children():
                _collect_layers(child)
    
    _collect_layers(model)
    target_layer = layers[layer_idx]
    
    def hook(module, input, output):
        activation['value'] = output.detach()
    
    return target_layer.register_forward_hook(hook)

def analyze_single_image(model, img_path, node_idx, activation, output_path, device):
    """分析单个图像的敏感区域"""
    # 加载图像
    img = Image.open(img_path).convert('RGB')
    input_tensor = preprocess(img).unsqueeze(0).to(device)
    
    # 创建掩码模板
    white_bg = torch.ones_like(input_tensor) * torch.tensor([
        (1.0 - 0.485)/0.229,
        (1.0 - 0.456)/0.224,
        (1.0 - 0.406)/0.225
    ], device=device).view(1, 3, 1, 1)
    
    max_activation = -float('inf')
    best_patch = None
    
    # 滑动窗口分析
    H, W = 224, 224
    size = Config.window_size
    for i in range(0, H - size + 1, Config.window_stride):
        for j in range(0, W - size + 1, Config.window_stride):
            # 创建局部掩码
            masked = white_bg.clone()
            masked[..., i:i+size, j:j+size] = input_tensor[..., i:i+size, j:j+size]
            
            # 前向传播
            with torch.no_grad():
                model(masked)
            
            # 获取激活值
            act = activation['value'][0, node_idx].mean().item()
            
            # 更新最佳区域
            if act > max_activation:
                max_activation = act
                best_patch = masked.clone()
    
    # 保存最佳结果
    if best_patch is not None:
        result_img = transforms.ToPILImage()(inv_normalize(best_patch.squeeze().cpu()))
        result_img.save(output_path)

def organize_files_by_label(file_path):
    """原始文件组织函数（保持原样）"""
    organized = {}
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 2:
                fname, label = parts[0], parts[1]
                if label not in organized:
                    organized[label] = []
                organized[label].append(fname.replace('.png', '.JPEG'))
    return organized

if __name__ == "__main__":
    # 初始化环境
    device = setup_device()
    node_mapping = load_node_mapping()
    
    # 加载模型
    model = models.resnet18(pretrained=True).to(device).eval()
    
    # 执行流程
    generate_adversarial_batch(model, device)
    analyze_activations(model, device, node_mapping)
    
    print("Processing completed.")

Using device: cuda

Generating FGSM adversarials...

Analyzing FGSM samples...


KeyboardInterrupt: 